# A.1 - RGB Only Training (V2)

**Experiment:** A.1 - RGB Baseline
**Input:** RGB images (3-channel)
**Objective:** Baseline detection using RGB only
**Classes:** 1 (fresh_fruit_bunch)

## Uniform Augmentation (All Experiments)
- translate: 0.1
- scale: 0.5
- fliplr: 0.5
- hsv_h: 0.0 (disabled)
- hsv_s: 0.0 (disabled)
- hsv_v: 0.0 (disabled)
- erasing: 0.0
- mosaic: 0.0
- mixup: 0.0

In [ ]:
# Cell 1: Environment Setup
import os
import sys
import torch
import numpy as np
from pathlib import Path

IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    BASE_PATH = Path('/kaggle/working')
    DATASET_PATH = Path('/kaggle/input/ffb-localization')
else:
    BASE_PATH = Path('D:/Work/Assisten Dosen/Anylabel/Experiments')
    DATASET_PATH = BASE_PATH / 'datasets' / 'ffb_localization'

RUNS_PATH = BASE_PATH / 'runs' / 'detect'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f'Running on: {"Kaggle" if IS_KAGGLE else "Local"}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# Cell 2: Install
!pip install -q ultralytics
from ultralytics import YOLO
import pandas as pd
import json
from datetime import datetime
import shutil
import gc
import time

In [ ]:
# Cell 3: Configuration
AUGMENT_PARAMS = {
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'hsv_h': 0.0,
    'hsv_s': 0.0,
    'hsv_v': 0.0,
    'erasing': 0.0,
    'mosaic': 0.0,
    'mixup': 0.0,
    'degrees': 0.0,
    'copy_paste': 0.0,
}

SEEDS = [42, 123, 456, 789, 101]
EXP_PREFIX = 'exp_a1_rgb_v2'
EPOCHS = 100
PATIENCE = 30
IMGSZ = 640
BATCH_SIZE = 16
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print(f'Experiment: A.1 RGB Only (V2)')
print(f'Seeds: {SEEDS}')
print(f'Epochs: {EPOCHS}')

In [ ]:
# Cell 4: Dataset Verification
for split in ['train', 'val', 'test']:
    img_dir = DATASET_PATH / 'images' / split
    lbl_dir = DATASET_PATH / 'labels' / split
    imgs = len(list(img_dir.glob('*.png'))) if img_dir.exists() else 0
    lbls = len(list(lbl_dir.glob('*.txt'))) if lbl_dir.exists() else 0
    print(f'{split}: {imgs} images, {lbls} labels')

yaml_content = f"""
path: {DATASET_PATH}
train: images/train
val: images/val
test: images/test
nc: 1
names: ['fresh_fruit_bunch']
"""

config_path = BASE_PATH / 'dataset_rgb_v2.yaml'
with open(config_path, 'w') as f:
    f.write(yaml_content)
print(f'Config: {config_path}')

In [ ]:
# Cell 5: Training Loop
results_all = {}

for idx, seed in enumerate(SEEDS, 1):
    print(f'\n=== Training Seed {seed} ({idx}/{len(SEEDS)}) ===')
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    model = YOLO('yolo11n.pt')
    
    try:
        results = model.train(
            data=str(config_path),
            epochs=EPOCHS,
            patience=PATIENCE,
            seed=seed,
            name=f'{EXP_PREFIX}_seed{seed}',
            project=str(RUNS_PATH),
            exist_ok=True,
            imgsz=IMGSZ,
            batch=BATCH_SIZE,
            device=DEVICE,
            **AUGMENT_PARAMS,
        )
        results_all[seed] = {'completed': True}
        print(f'Seed {seed} done!')
    except Exception as e:
        print(f'Seed {seed} failed: {e}')
        results_all[seed] = {'error': str(e)}
    
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
# Cell 6: Evaluation
results_dict = {}

for seed in SEEDS:
    model_path = RUNS_PATH / f'{EXP_PREFIX}_seed{seed}' / 'weights' / 'best.pt'
    if not model_path.exists():
        continue
    
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test', device=DEVICE)
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr,
    }
    print(f'Seed {seed}: mAP50={metrics.box.map50:.4f}')
    
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
# Cell 7: Summary
if results_dict:
    df = pd.DataFrame(results_dict).T
    df.index.name = 'Seed'
    
    print('A.1 RGB ONLY (V2) - RESULTS\n')
    print(df.to_string())
    
    print(f'\nMean:')
    for col in df.columns:
        print(f'  {col}: {df[col].mean():.4f} ± {df[col].std():.4f}')
    
    print(f'\nBest Seed: {df["mAP50"].idxmax()}')

In [ ]:
# Cell 8: Save Results
output_file = KAGGLE_OUTPUT / 'a1_rgb_v2_results.txt'

with open(output_file, 'w') as f:
    f.write('A.1 RGB Only (V2) Results\n')
    f.write(f'Generated: {datetime.now()}\n\n')
    if results_dict:
        f.write(df.to_string())
        f.write('\n\nSummary:\n')
        for col in df.columns:
            f.write(f'{col}: {df[col].mean():.4f} ± {df[col].std():.4f}\n')

print(f'Results saved: {output_file}')

In [ ]:
# Cell 9: Create Archives
if RUNS_PATH.exists():
    shutil.make_archive(str(BASE_PATH / 'a1_rgb_v2_runs'), 'zip', RUNS_PATH)
    print('✓ a1_rgb_v2_runs.zip created')

shutil.make_archive(str(BASE_PATH / 'a1_rgb_v2_output'), 'zip', KAGGLE_OUTPUT)
print('✓ a1_rgb_v2_output.zip created')